## 3.1 EntropyMeasure

The instance `EntropyMeasure` is actually contained 2 case, one is `EntropyHadamardTest` using Hadamard test, and the other is `EntropyRandomizedMeasure` using randomized measure, the default case.

Both of them are used to calculate the entropy of a quantum state. But their calculation and post-processing are totally different.


### 3.1.1 Randomized Measure

This method is based on [Probing Rényi entanglement entropy via randomized measurements](https://www.science.org/doi/abs/10.1126/science.aau4963) with deplorizing error mitigation by [Simple mitigation of global depolarizing errors in quantum simulations](https://link.aps.org/doi/10.1103/PhysRevE.104.035309).


---

### a. `randomized_entangled_entropy`

<a id='randomized_entangled_entropy'></a>


Here, we assume you already have the counts from your own calculation. You can call the function `randomized_entangled_entropy` from `qurry.process.randomized_measure` to get the entropy.

The following is the arguments of the function:

```python
def randomized_entangled_entropy(
    shots: int,
    counts: list[dict[str, int]],
    degree: Optional[Union[tuple[int, int], int]],
    measure: Optional[tuple[int, int]] = None,
    backend: PostProcessingBackendLabel = DEFAULT_PROCESS_BACKEND,
    workers_num: Optional[int] = None,
    pbar: Optional[tqdm.tqdm] = None,
) -> RandomizedEntangledEntropyComplex:
    """Calculate entangled entropy.

    Args:
        shots (int):
            Shots of the counts.
        counts (list[dict[str, int]]):
            Counts from randomized measurement results.
        degree (Optional[Union[tuple[int, int], int]]):
            The range of partition.
        measure (Optional[tuple[int, int]], optional):
            The range that implemented the measuring gate.
            If not specified, then use all qubits.
            This will affect the range of partition
            when you not implement the measuring gate on all qubit.
            Defaults to None.
        backend (PostProcessingBackendLabel, optional):
            Backend for the post-processing.
            Defaults to DEFAULT_PROCESS_BACKEND.
        workers_num (Optional[int], optional):
            Number of multi-processing workers, it will be ignored if backend is Rust.
            if sets to 1, then disable to using multi-processing;
            if not specified, then use the number of all cpu counts by `os.cpu_count()`.
            This only works for Python and Cython backend.
            Defaults to None.
        pbar (Optional[tqdm.tqdm], optional):
            The progress bar API, you can use put a :cls:`tqdm` object here.
            This function will update the progress bar description.
            Defaults to None.

    Returns:
        RandomizedEntangledEntropyComplex:
            A dictionary contains purity, entropy,
            a list of each overlap, puritySD, degree,
            actual measure range, bitstring range and more.
    """
```

The return `dict` will contained following variables:

```python
class RandomizedEntangledEntropyComplex(TypedDict):
    """The result of the analysis."""

    purity: Union[np.float64, float]
    """The purity of the system."""
    entropy: Union[np.float64, float]
    """The entropy of the system."""
    purityCells: Union[dict[int, np.float64], dict[int, float]]
    """The purity of each cell."""
    puritySD: Union[np.float64, float]
    """The standard deviation of the purity."""
    entropySD: Union[np.float64, float]
    """The standard deviation of the entropy."""
    degree: Optional[Union[tuple[int, int], int]]
    """The range of partition."""
    measureActually: Union[tuple[int, int], tuple[()]]
    """The range of partition refer to all qubits."""
    bitStringRange: Union[tuple[int, int], tuple[()]]
    """The range of partition on the bitstring."""
    countsNum: int
    """The number of counts."""
```

This function will return a dictionary that contains the entropy, purity, and some other information. In this function, only `shots`, `counts`, and `degree` are required. The others are optional and we do not recommend to change them unless you are familiar with the source code.


---

#### Dummy Data


In [1]:
from qurry.capsule import quickRead

easy_dummy: dict[str, dict[str, int]] = quickRead("./easy-dummy.json")
large_dummy_list = [easy_dummy["0"] for _ in range(100)]

---

#### Simple Example


In [2]:
from qurry.process.randomized_measure import (
    randomized_entangled_entropy,
)

test_result_1_1_1 = randomized_entangled_entropy(4096, large_dummy_list, 6)

In [3]:
from pprint import pprint

print("| result of randomized_entangled_entropy except for purityCells")
pprint({k: v for k, v in test_result_1_1_1.items() if k != "purityCells"})
# "purityCells" is too long we skip it here

print("| result of randomized_entangled_entropy[purityCells]")
print(test_result_1_1_1["purityCells"][0])
print(test_result_1_1_1["purityCells"][1])

| result of randomized_entangled_entropy except for purityCells
{'bitStringRange': (2, 8),
 'countsNum': 100,
 'degree': 6,
 'entropy': -0.0878606530863832,
 'entropySD': 0.0,
 'measureActually': (2, 8),
 'purity': 1.0627930164337158,
 'puritySD': 0.0}
| result of randomized_entangled_entropy[purityCells]
1.0627930164337158
1.0627930164337158


---

#### Integration wit your own progress bar


In [4]:
from tqdm import tqdm
import time

all_counts_progress_01 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)

test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_01:
    test_result_1_1_2.append(
        randomized_entangled_entropy(
            tmp_shot, tmp_counts, tmp_partition, pbar=all_counts_progress_01
        )
    )
    time.sleep(0.5)

| Calculate specific degree (3, -2). - 00:03 < 00:00 


---

#### Using Python backend

It will be slow. Yoy better think twice before using it.


In [5]:
all_counts_progress_02 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)


test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_02:
    test_result_1_1_2.append(
        randomized_entangled_entropy(
            tmp_shot,
            tmp_counts,
            tmp_partition,
            pbar=all_counts_progress_02,
            backend="Python",
        )
    )
    time.sleep(0.5)

| Calculate specific degree (3, -2). - 00:05 < 00:00 


---

### Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [6]:
from qurry.process.status import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.7.4.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ......... Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------
by <Hoshi>